CONTENT BASED MOVIE RECOMMENDER SYSTEM USING SPARK

In [0]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [0]:
import sparknlp
#create or get Spark Session
spark = sparknlp.start()

Reading the data

In [0]:
movies = spark.read.table("tmdb_5000_movies_csv")
movies.limit(2).display()

budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 9685, ""name"": ""futuristic""}, {""id"": 9840, ""name"": ""romance""}, {""id"": 9882, ""name"": ""space""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10148, ""name"": ""tribe""}, {""id"": 10158, ""name"": ""alien planet""}, {""id"": 10987, ""name"": ""cgi""}, {""id"": 11399, ""name"": ""marine""}, {""id"": 13065, ""name"": ""soldier""}, {""id"": 14643, ""name"": ""battle""}, {""id"": 14720, ""name"": ""love affair""}, {""id"": 165431, ""name"": ""anti war""}, {""id"": 193554, ""name"": ""power relations""}, {""id"": 206690, ""name"": ""mind and soul""}, {""id"": 209714, ""name"": ""3d""}]",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.43758,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id"": 574}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-12-10T00:00:00.000+0000,null,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life""}, {""id"": 2052, ""name"": ""traitor""}, {""id"": 2580, ""name"": ""shipwreck""}, {""id"": 2660, ""name"": ""strong woman""}, {""id"": 3799, ""name"": ""ship""}, {""id"": 5740, ""name"": ""alliance""}, {""id"": 5941, ""name"": ""calypso""}, {""id"": 6155, ""name"": ""afterlife""}, {""id"": 6211, ""name"": ""fighter""}, {""id"": 12988, ""name"": ""pirate""}, {""id"": 157186, ""name"": ""swashbuckler""}, {""id"": 179430, ""name"": ""aftercreditsstinger""}]",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.08261,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""name"": ""Second Mate Productions"", ""id"": 19936}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19T00:00:00.000+0000,961000000,169,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [0]:
credits = spark.read.table("default.tmdb_5000_credits_csv")
credits.limit(1).display()

movie_id title cast crew 19995 Avatar [{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 2, "id": 59231, "name": "Joel David Moore", "order": 6}, {"cast_id": 9, "character": "Moat", "credit_id": "52fe48009251416c750ac9e5", "gender": 1, "id": 30485, "name": "CCH Pounder", "order": 7}, {"cast_id": 11, "character": "Eytukan", "credit_id": "52fe48009251416c750ac9ed", "gender": 2, "id": 15853, "name": "Wes Studi", "order": 8}, {"cast_id": 10, "character": "Tsu'Tey", "credit_id": "52fe48009251416c750ac9e9", "gender": 2, "id": 10964, "name": "Laz Alonso", "order": 9}, {"cast_id": 12, "character": "Dr. Max Patel", "credit_id": "52fe48009251416c750ac9f1", "gender": 2, "id": 95697, "name": "Dileep Rao", "order": 10}, {"cast_id": 13, "character": "Lyle Wainfleet", "credit_id": "52fe48009251416c750ac9f5", "gender": 2, "id": 98215, "name": "Matt Gerald", "order": 11}, {"cast_id": 32, "character": "Private Fike", "credit_id": "52fe48009251416c750aca5b", "gender": 2, "id": 154153, "name": "Sean Anthony Moran", "order": 12}, {"cast_id": 33, "character": "Cryo Vault Med Tech", "credit_id": "52fe48009251416c750aca5f", "gender": 2, "id": 397312, "name": "Jason Whyte", "order": 13}, {"cast_id": 34, "character": "Venture Star Crew Chief", "credit_id": "52fe48009251416c750aca63", "gender": 2, "id": 42317, "name": "Scott Lawrence", "order": 14}, {"cast_id": 35, "character": "Lock Up Trooper", "credit_id": "52fe48009251416c750aca67", "gender": 2, "id": 986734, "name": "Kelly Kilgour", "order": 15}, {"cast_id": 36, "character": "Shuttle Pilot", "credit_id": "52fe48009251416c750aca6b", "gender": 0, "id": 1207227, "name": "James Patrick Pitt", "order": 16}, {"cast_id": 37, "character": "Shuttle Co-Pilot", "credit_id": "52fe48009251416c750aca6f", "gender": 0, "id": 1180936, "name": "Sean Patrick Murphy", "order": 17}, {"cast_id": 38, "character": "Shuttle Crew Chief", "credit_id": "52fe48009251416c750aca73", "gender": 2, "id": 1019578, "name": "Peter Dillon", "order": 18}, {"cast_id": 39, "character": "Tractor Operator / Troupe", "credit_id": "52fe48009251416c750aca77", "gender": 0, "id": 91443, "name": "Kevin Dorman", "order": 19}, {"cast_id": 40, "character": "Dragon Gunship Pilot", "credit_id": "52fe48009251416c750aca7b", "gender": 2, "id": 173391, "name": "Kelson Henderson", "order": 20}, {"cast_id": 41, "character": "Dragon Gunship Gunner", "credit_id": "52fe48009251416c750aca7f", "gender": 0, "id": 1207236, "name": "David Van Horn", "order": 21}, {"cast_id": 42, "character": "Dragon Gunship Navigator", "credit_id": "52fe48009251416c750aca83", "gender": 0, "id": 215913, "name": "Jacob Tomuri", "order": 22}, {"cast_id": 43, "character": "Suit #1", "credit_id": "52fe48009251416c750aca87", "gender": 0, "id": 143206, "name": "Michael Blain-Rozgay", "order": 23}, {"cast_id": 44, "character": "Suit #2", "credit_id": "52fe48009251416c750aca8b", "gender": 2, "id": 169676, "name": "Jon Curry", "order": 24}, {"cast_id": 46, "character": "Ambient Room Tech", "credit_id": "52fe48009251416c750aca8f", "gender": 0, "id": 1048610, "nam

STEP 1: DATA PREPROCESSING

Merging the two dataframes to work on a single dataframe

In [0]:
df = movies.join(credits, ["title"])
df.limit(1).display()

title budget genres homepage id keywords original_language original_title overview popularity production_companies production_countries release_date revenue runtime spoken_languages status tagline vote_average vote_count movie_id cast crew Avatar 237000000 [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}] http://www.avatarmovie.com/ 19995 [{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}] en Avatar In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. 150.43758 [{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}] [{"iso_3166_1": "US", "name": "United States of America"}, {"iso_3166_1": "GB", "name": "United Kingdom"}] 2009-12-10T00:00:00.000+0000 null 162 [{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "es", "name": "Espa\u00f1ol"}] Released Enter the World of Pandora. 7.2 11800 19995 [{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 2, "id": 59231, "name": "Joel David Moore", "order": 6}, {"cast_id": 9, "character": "Moat", "credit_id": "52fe48009251416c750ac9e5", "gender": 1, "id": 30485, "name": "CCH Pounder", "order": 7}, {"cast_id": 11, "character": "Eytukan", "credit_id": "52fe48009251416c750ac9ed", "gender": 2, "id": 15853, "name": "Wes Studi", "order": 8}, {"cast_id": 10, "character": "Tsu'Tey", "credit_id": "52fe48009251416c750ac9e9", "gender": 2, "id": 10964, "name": "Laz Alonso", "order": 9}, {"cast_id": 12, "character": "Dr. Max Patel", "credit_id": "52fe48009251416c750ac9f1", "gender": 2, "id": 95697, "name": "Dileep Rao", "order": 10}, {"cast_id": 13, "character": "Lyle Wainfleet", "credit_id": "52fe48009251416c750ac9f5", "gender": 2, "id": 98215, "name": "Matt Gerald", "order": 11}, {"cast_id": 32, "character": "Private Fike", "credit_id": "52fe48009251416c750aca5b", "gender": 2, "id": 154153, "name": "Sean Anthony Moran", "order": 12}, {"cast_id": 33, "character": "Cryo Vault Med Tech", "credit_id": "52fe48009251416c750aca5f", "gender": 2, "id": 397312, "name": "Jason Whyte", "order": 13}, {"cast_id": 34, "character": "Venture Star Crew Chief", "credi

All the Data has been gathered into a single dataframe

In [0]:
df.count()

Out[9]: 4806

RECOMMENDATION SYSTEM ARE OF 2 TYPES
1. CONTENT BASED RECOMMENDATION SYSTEM: Based on similarity of Content
2. COLLABORATIVE FILTERING: Based on similar viewing users

The method chosen here is CONTENT BASED RECOMMENDER SYSTEM.

After initial scanning we eliminate some features which will not be helpful to us such as budget, production_countries, run_time, spoken languages, status, vote_count, original_language, original_title, movie_id

In [0]:
df = df.select( "id", "title", "genres", "keywords", "overview", "cast", "crew")
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|              genres|            keywords|            overview|                cast|                crew|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|[{"id": 28, "name...|[{"id": 1463, "na...|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|
|   285|Pirates of the Ca...|[{"id": 12, "name...|[{"id": 270, "nam...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|
|206647|             Spectre|[{"id": 28, "name...|[{"id": 470, "nam...|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|
| 49026|The Dark Knight R...|[{"id": 28, "name...|[{"id": 849, "nam...|Following the dea...|[{"cast_id": 2, "...|[{"credit_id": "5...|
| 49529|         John Carter|[{"id": 28, "name...|[{"id

CHECKING FOR NULL VALUES

In [0]:
import pyspark.sql.functions as F
df.select([F.count(F.when( F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-----+------+--------+--------+----+----+
| id|title|genres|keywords|overview|cast|crew|
+---+-----+------+--------+--------+----+----+
|  0|    0|     0|       0|       3|   0|   0|
+---+-----+------+--------+--------+----+----+



Dropping rows where null value is present

In [0]:
df = df.dropna(subset= "overview", how = "any")
df.select([F.count(F.when( F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-----+------+--------+--------+----+----+
| id|title|genres|keywords|overview|cast|crew|
+---+-----+------+--------+--------+----+----+
|  0|    0|     0|       0|       0|   0|   0|
+---+-----+------+--------+--------+----+----+



EXTRACTING THE LIST OF GENRES FROM GENRES COLUMN

In [0]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, udf
from pyspark.sql.types import StringType, ArrayType, MapType, IntegerType, LongType
import ast

In [0]:
genre_list = udf(lambda a: ast.literal_eval(a), ArrayType(MapType(StringType(), StringType())))
df = df.withColumn("genre", genre_list(F.col("genres")))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|              genres|            keywords|            overview|                cast|                crew|               genre|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|[{"id": 28, "name...|[{"id": 1463, "na...|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[{name -> Action,...|
|   285|Pirates of the Ca...|[{"id": 12, "name...|[{"id": 270, "nam...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[{name -> Adventu...|
|206647|             Spectre|[{"id": 28, "name...|[{"id": 470, "nam...|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|[{name -> Action,...|
| 49026|The Dark Knight R...|[{"id": 28, "name...|[{"id": 849, "

In [0]:
def convert(list):
    l = []
    for dict in list:
        l.append(dict["name"])
    return(l)
#UDF
convert_udf = udf(lambda a: convert(a), ArrayType(StringType()))

In [0]:
df = df.withColumn("movie_genre", convert_udf((F.col("genre"))))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|              genres|            keywords|            overview|                cast|                crew|               genre|         movie_genre|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|[{"id": 28, "name...|[{"id": 1463, "na...|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[{name -> Action,...|[Action, Adventur...|
|   285|Pirates of the Ca...|[{"id": 12, "name...|[{"id": 270, "nam...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[{name -> Adventu...|[Adventure, Fanta...|
|206647|             Spectre|[{"id": 28, "name...|[{"id": 470, "nam...|A cryptic message...|[{"cast_id": 1, "...|[{

We have successfully extracted the movie genre!!

In [0]:
df = df.drop("genre", "genres")
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            keywords|            overview|                cast|                crew|         movie_genre|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|[{"id": 1463, "na...|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[Action, Adventur...|
|   285|Pirates of the Ca...|[{"id": 270, "nam...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[Adventure, Fanta...|
|206647|             Spectre|[{"id": 470, "nam...|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|[Action, Adventur...|
| 49026|The Dark Knight R...|[{"id": 849, "nam...|Following the dea...|[{"cast_id": 2, "...|[{"credit_id": "5...|[Action, Crime, D...|
| 49529|         John Carter|[{"id": 818, "nam...|John 

EXTRACTING KEYWORDS FROM KEYWORDS FEATURE

In [0]:
keywords_list = udf(lambda a: ast.literal_eval(a), ArrayType(MapType(StringType(), StringType())))
df = df.withColumn("key_words", keywords_list(F.col("keywords")))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            keywords|            overview|                cast|                crew|         movie_genre|           key_words|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|[{"id": 1463, "na...|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[Action, Adventur...|[{name -> culture...|
|   285|Pirates of the Ca...|[{"id": 270, "nam...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[Adventure, Fanta...|[{name -> ocean, ...|
|206647|             Spectre|[{"id": 470, "nam...|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|[Action, Adventur...|[{name -> spy, id...|
| 49026|The Dark Knight R...|[{"id": 849, "nam...|Following the 

In [0]:
df = df.withColumn("keyword", convert_udf(F.col("key_words")))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            keywords|            overview|                cast|                crew|         movie_genre|           key_words|             keyword|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|[{"id": 1463, "na...|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[Action, Adventur...|[{name -> culture...|[culture clash, f...|
|   285|Pirates of the Ca...|[{"id": 270, "nam...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[Adventure, Fanta...|[{name -> ocean, ...|[ocean, drug abus...|
|206647|             Spectre|[{"id": 470, "nam...|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|[A

In [0]:
df = df.drop("keywords", "key_words")
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            overview|                cast|                crew|         movie_genre|             keyword|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[Action, Adventur...|[culture clash, f...|
|   285|Pirates of the Ca...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[Adventure, Fanta...|[ocean, drug abus...|
|206647|             Spectre|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|[Action, Adventur...|[spy, based on no...|
| 49026|The Dark Knight R...|Following the dea...|[{"cast_id": 2, "...|[{"credit_id": "5...|[Action, Crime, D...|[dc comics, crime...|
| 49529|         John Carter|John Carter is a ...|[{"ca

EXTRACTING THE TOP 3 ACTORS OF THE MOVIE FROM THE CAST COLUMN.                      
Viewers often watch/prefer to watch movies of their favourite actors in lead

In [0]:
cast_list = udf(lambda a: ast.literal_eval(a), ArrayType(MapType(StringType(), StringType())))
df = df.withColumn("movie_cast", cast_list(F.col("cast")))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            overview|                cast|                crew|         movie_genre|             keyword|          movie_cast|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[Action, Adventur...|[culture clash, f...|[{character -> Ja...|
|   285|Pirates of the Ca...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[Adventure, Fanta...|[ocean, drug abus...|[{character -> Ca...|
|206647|             Spectre|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|[Action, Adventur...|[spy, based on no...|[{character -> Ja...|
| 49026|The Dark Knight R...|Following the dea...|[{"cast_id": 2

In [0]:
def actor_name(list):
    l = []
    i = 0
    for dict in list:
        if i != 3:
            l.append(dict["name"])
            i += 1
        else:
            break;
    return(l)

In [0]:
cast_name = udf(lambda a: actor_name(a), ArrayType(StringType()))

In [0]:
df = df.withColumn("cast_name", cast_name((F.col("movie_cast"))))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            overview|                cast|                crew|         movie_genre|             keyword|          movie_cast|           cast_name|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|In the 22nd centu...|[{"cast_id": 242,...|[{"credit_id": "5...|[Action, Adventur...|[culture clash, f...|[{character -> Ja...|[Sam Worthington,...|
|   285|Pirates of the Ca...|Captain Barbossa,...|[{"cast_id": 4, "...|[{"credit_id": "5...|[Adventure, Fanta...|[ocean, drug abus...|[{character -> Ca...|[Johnny Depp, Orl...|
|206647|             Spectre|A cryptic message...|[{"cast_id": 1, "...|[{"credit_id": "5...|[Action, Adventur...|[s

In [0]:
df = df.drop("cast", "movie_cast")
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            overview|                crew|         movie_genre|             keyword|           cast_name|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|In the 22nd centu...|[{"credit_id": "5...|[Action, Adventur...|[culture clash, f...|[Sam Worthington,...|
|   285|Pirates of the Ca...|Captain Barbossa,...|[{"credit_id": "5...|[Adventure, Fanta...|[ocean, drug abus...|[Johnny Depp, Orl...|
|206647|             Spectre|A cryptic message...|[{"credit_id": "5...|[Action, Adventur...|[spy, based on no...|[Daniel Craig, Ch...|
| 49026|The Dark Knight R...|Following the dea...|[{"credit_id": "5...|[Action, Crime, D...|[dc comics, crime...|[Christian Bale, ...|
| 49529|         John Carter|John Carter is a ...|[{"cr

FETCHING DIRECTOR NAME FROM CREW FEATURE

In [0]:
crew_list = udf(lambda a: ast.literal_eval(a), ArrayType(MapType(StringType(), StringType())))
df = df.withColumn("movie_crew", crew_list(F.col("crew")))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|            overview|                crew|         movie_genre|             keyword|           cast_name|          movie_crew|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 19995|              Avatar|In the 22nd centu...|[{"credit_id": "5...|[Action, Adventur...|[culture clash, f...|[Sam Worthington,...|[{gender -> 0, cr...|
|   285|Pirates of the Ca...|Captain Barbossa,...|[{"credit_id": "5...|[Adventure, Fanta...|[ocean, drug abus...|[Johnny Depp, Orl...|[{gender -> 2, cr...|
|206647|             Spectre|A cryptic message...|[{"credit_id": "5...|[Action, Adventur...|[spy, based on no...|[Daniel Craig, Ch...|[{gender -> 2, cr...|
| 49026|The Dark Knight R...|Following the dea...|[{"credit_id":

In [0]:
def director_name(list):
    l = []
    for dict in list:
        if dict["job"] == "Director":
            l.append(dict["name"])
    return(l)

In [0]:
director = udf(lambda a: director_name(a), ArrayType(StringType()))
df = df.withColumn("director", director((F.col("movie_crew"))))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|    id|               title|            overview|                crew|         movie_genre|             keyword|           cast_name|          movie_crew|           director|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
| 19995|              Avatar|In the 22nd centu...|[{"credit_id": "5...|[Action, Adventur...|[culture clash, f...|[Sam Worthington,...|[{gender -> 0, cr...|    [James Cameron]|
|   285|Pirates of the Ca...|Captain Barbossa,...|[{"credit_id": "5...|[Adventure, Fanta...|[ocean, drug abus...|[Johnny Depp, Orl...|[{gender -> 2, cr...|   [Gore Verbinski]|
|206647|             Spectre|A cryptic message...|[{"credit_id": "5...|[Action, Adventur...|[spy, based on no...|[Daniel

In [0]:
df = df.drop("crew", "movie_crew")
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|    id|               title|            overview|         movie_genre|             keyword|           cast_name|           director|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
| 19995|              Avatar|In the 22nd centu...|[Action, Adventur...|[culture clash, f...|[Sam Worthington,...|    [James Cameron]|
|   285|Pirates of the Ca...|Captain Barbossa,...|[Adventure, Fanta...|[ocean, drug abus...|[Johnny Depp, Orl...|   [Gore Verbinski]|
|206647|             Spectre|A cryptic message...|[Action, Adventur...|[spy, based on no...|[Daniel Craig, Ch...|       [Sam Mendes]|
| 49026|The Dark Knight R...|Following the dea...|[Action, Crime, D...|[dc comics, crime...|[Christian Bale, ...|[Christopher Nolan]|
| 49529|         John Carter|John Carter is a ...|[Action, Adv

CONVERTING OVERVIEW COLUMN FROM STRING TO LIST OF WORDS

In [0]:
list_udf = udf(lambda a: a.split(" "), ArrayType(StringType()))

In [0]:
df = df.withColumn("overview", list_udf(F.col("overview")))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|    id|               title|            overview|         movie_genre|             keyword|           cast_name|           director|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
| 19995|              Avatar|[In, the, 22nd, c...|[Action, Adventur...|[culture clash, f...|[Sam Worthington,...|    [James Cameron]|
|   285|Pirates of the Ca...|[Captain, Barboss...|[Adventure, Fanta...|[ocean, drug abus...|[Johnny Depp, Orl...|   [Gore Verbinski]|
|206647|             Spectre|[A, cryptic, mess...|[Action, Adventur...|[spy, based on no...|[Daniel Craig, Ch...|       [Sam Mendes]|
| 49026|The Dark Knight R...|[Following, the, ...|[Action, Crime, D...|[dc comics, crime...|[Christian Bale, ...|[Christopher Nolan]|
| 49529|         John Carter|[John, Carter, is...|[Action, Adv

As our aim is to create tags, we will have to remove spaces between compound words if any so that the individual word in a compound word is not treated as a separate tag

In [0]:
remove_space = udf(lambda a: [i.replace(" ", "") for i in a], ArrayType(StringType()))
for column in ["movie_genre", "keyword", "cast_name", "director"]:
    df = df.withColumn(column, remove_space(F.col(column)))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|    id|               title|            overview|         movie_genre|             keyword|           cast_name|          director|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
| 19995|              Avatar|[In, the, 22nd, c...|[Action, Adventur...|[cultureclash, fu...|[SamWorthington, ...|    [JamesCameron]|
|   285|Pirates of the Ca...|[Captain, Barboss...|[Adventure, Fanta...|[ocean, drugabuse...|[JohnnyDepp, Orla...|   [GoreVerbinski]|
|206647|             Spectre|[A, cryptic, mess...|[Action, Adventur...|[spy, basedonnove...|[DanielCraig, Chr...|       [SamMendes]|
| 49026|The Dark Knight R...|[Following, the, ...|[Action, Crime, D...|[dccomics, crimef...|[ChristianBale, M...|[ChristopherNolan]|
| 49529|         John Carter|[John, Carter, is...|[Action, Adventur..

In [0]:
df = df.withColumn("tags", F.concat(F.col("overview"),F.col("movie_genre"),F.col("keyword"),F.col("cast_name"),F.col("director")))
df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|    id|               title|            overview|         movie_genre|             keyword|           cast_name|          director|                tags|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
| 19995|              Avatar|[In, the, 22nd, c...|[Action, Adventur...|[cultureclash, fu...|[SamWorthington, ...|    [JamesCameron]|[In, the, 22nd, c...|
|   285|Pirates of the Ca...|[Captain, Barboss...|[Adventure, Fanta...|[ocean, drugabuse...|[JohnnyDepp, Orla...|   [GoreVerbinski]|[Captain, Barboss...|
|206647|             Spectre|[A, cryptic, mess...|[Action, Adventur...|[spy, basedonnove...|[DanielCraig, Chr...|       [SamMendes]|[A, cryptic, mess...|
| 49026|The Dark Knight R...|[Following, the, ...|[Action, Crime, D...|[dcco

In [0]:
new_df = df.select("id", "title", "tags")
new_df.show(5)

+------+--------------------+--------------------+
|    id|               title|                tags|
+------+--------------------+--------------------+
| 19995|              Avatar|[In, the, 22nd, c...|
|   285|Pirates of the Ca...|[Captain, Barboss...|
|206647|             Spectre|[A, cryptic, mess...|
| 49026|The Dark Knight R...|[Following, the, ...|
| 49529|         John Carter|[John, Carter, is...|
+------+--------------------+--------------------+
only showing top 5 rows



Remove stop words

In [0]:
from pyspark.ml.feature import CountVectorizer, StopWordsRemover
swr = StopWordsRemover()
swr.setInputCol("tags").setOutputCol("tags_transformed")
new_df = swr.transform(new_df)
new_df = new_df.drop("tags")
new_df.show(5)

+------+--------------------+--------------------+
|    id|               title|    tags_transformed|
+------+--------------------+--------------------+
| 19995|              Avatar|[22nd, century,, ...|
|   285|Pirates of the Ca...|[Captain, Barboss...|
|206647|             Spectre|[cryptic, message...|
| 49026|The Dark Knight R...|[Following, death...|
| 49529|         John Carter|[John, Carter, wa...|
+------+--------------------+--------------------+
only showing top 5 rows



In [0]:
concatenate = udf(lambda x: " ".join(x))

In [0]:
new_df = new_df.withColumn("tags_transformed", concatenate(F.col("tags_transformed")).astype(StringType()))
new_df.show(5)

+------+--------------------+--------------------+
|    id|               title|    tags_transformed|
+------+--------------------+--------------------+
| 19995|              Avatar|22nd century, par...|
|   285|Pirates of the Ca...|Captain Barbossa,...|
|206647|             Spectre|cryptic message B...|
| 49026|The Dark Knight R...|Following death D...|
| 49529|         John Carter|John Carter war-w...|
+------+--------------------+--------------------+
only showing top 5 rows



STEMMING: Stemming is a process to transform a word into its root word so that all the related words in different forms like adjective/noun can be treated as the same tag

In [0]:
import sparknlp
from sparknlp.annotator import Stemmer

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler() \
    .setInputCol("tags_transformed") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

pipeline = Pipeline().setStages([
    documentAssembler,
    tokenizer,
    stemmer
])

In [0]:
new_df = pipeline.fit(new_df).transform(new_df).selectExpr("id", "title", "stem.result")
new_df.show(5)

+------+--------------------+--------------------+
|    id|               title|              result|
+------+--------------------+--------------------+
| 19995|              Avatar|[22nd, centuri, ,...|
|   285|Pirates of the Ca...|[captain, barboss...|
|206647|             Spectre|[cryptic, messag,...|
| 49026|The Dark Knight R...|[follow, death, d...|
| 49529|         John Carter|[john, carter, wa...|
+------+--------------------+--------------------+
only showing top 5 rows



In [0]:
cv = CountVectorizer(inputCol= "result", outputCol="tag_vector", vocabSize= 5000)

In [0]:
model = cv.fit(new_df)
new_df = model.transform(new_df)
new_df.show(5)

+------+--------------------+--------------------+--------------------+
|    id|               title|              result|          tag_vector|
+------+--------------------+--------------------+--------------------+
| 19995|              Avatar|[22nd, centuri, ,...|(5000,[0,1,5,7,8,...|
|   285|Pirates of the Ca...|[captain, barboss...|(5000,[0,1,5,8,11...|
|206647|             Spectre|[cryptic, messag,...|(5000,[0,1,5,8,10...|
| 49026|The Dark Knight R...|[follow, death, d...|(5000,[0,1,2,4,5,...|
| 49529|         John Carter|[john, carter, wa...|(5000,[0,1,5,8,14...|
+------+--------------------+--------------------+--------------------+
only showing top 5 rows



We have obtained a sparse vector, we'll need to convert it into a dense vector

In [0]:
from pyspark.mllib.linalg import VectorUDT, DenseVector, ArrayType, VectorUDT, Vectors
from pyspark.mllib.linalg.distributed import RowMatrix, CoordinateMatrix, BlockMatrix

In [0]:
sparse_values = udf(lambda v: Vectors.dense(v), VectorUDT())
vector_df = new_df.select(sparse_values("tag_vector")).withColumnRenamed('<lambda>(tag_vector)', "tag_vector")

Preparing the movie dataframe

In [0]:
from pyspark.sql.window import Window
new_df = new_df.withColumn("idx", F.monotonically_increasing_id())

# Create the window specification
w = Window.orderBy("idx")

# Use row number with the window specification
new_df = new_df.withColumn("index", F.row_number().over(w))
new_df = new_df.withColumn("index", F.col("index") - 1)
# Drop the created increasing data column
new_df = new_df.drop("idx", "result", "tag_vector")
new_df.cache()

Out[46]: DataFrame[id: int, title: string, index: int]

Preparing the similarity dataframe,

In [0]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry, BlockMatrix
mat = RowMatrix(vector_df)
cm = CoordinateMatrix(
    mat.rows.zipWithIndex().flatMap(
        lambda x: [(x[1], j, v) for j, v in enumerate(x[0])]
    )
)
sim = cm.transpose().toRowMatrix().columnSimilarities()

In [0]:
entryRDD= sim.entries.union(sim.transpose().entries).union(sc.parallelize([(i,i,1.0) for i in range(sim.numCols())]))
entryRDD.cache()

Out[48]: UnionRDD[414] at union at NativeMethodAccessorImpl.java:0

In [0]:
def recommend_movie(movie):
    index = new_df.select("index").where(F.col("title") == movie).head()[0]
    recommended_movie_id = entryRDD.toDF().where(F.col("i") == index).orderBy(F.col("value").desc()).select("j").collect()[1:6]
    return(recommended_movie_id)
    

In [0]:
new_df.select("index", "id", "title").show(5)

+-----+------+--------------------+
|index|    id|               title|
+-----+------+--------------------+
|    0| 19995|              Avatar|
|    1|   285|Pirates of the Ca...|
|    2|206647|             Spectre|
|    3| 49026|The Dark Knight R...|
|    4| 49529|         John Carter|
+-----+------+--------------------+
only showing top 5 rows



In [0]:
sim_df = entryRDD.toDF()

In [0]:
sim_df.show(5)

+----+----+-------------------+
|   i|   j|              value|
+----+----+-------------------+
| 233|1265|  0.270232002768764|
|2959|3989|0.04351941398892446|
| 372|2024|0.04393747751637468|
|3471|4724|0.26703560975577234|
| 918|1768|0.43039022414354194|
+----+----+-------------------+
only showing top 5 rows



In [0]:
import pickle
pickle.dump(sim_df, open('similarity.pickle', 'wb'))
pickle.dump(new_df, open('movies.pickle', 'wb'))